In [1]:
!pip install --upgrade --quiet pip
!pip install --quiet haystack-ai
print('pip install haystack complete.')

pip install haystack complete.


Let's get the demo from here: https://docs.haystack.deepset.ai/docs/get_started working with a HuggingFace pipeline instead of OpenAI. 

First let's create some data.

In [2]:
import arrow

time_start = arrow.now()
documents = [
    'My name is Jean and I live in Paris.', 
    'My name is Mark and I live in Berlin.', 
    'My name is Giorgio and I live in Rome.'
]
print('built documents in {}'.format(arrow.now() - time_start))

built documents in 0:00:00.000198


Let's build a document store and populate it with our data.

In [3]:
import arrow
from haystack import Document
from haystack.document_stores.in_memory import InMemoryDocumentStore


time_start = arrow.now()
document_store = InMemoryDocumentStore()
document_store.write_documents([Document(content=content) for content in documents])
print('built/populated document store in {}'.format(arrow.now() - time_start))


built/populated document store in 0:00:00.000478


In [4]:
import arrow
from haystack import Pipeline
from haystack.components.builders.answer_builder import AnswerBuilder
from haystack.components.builders.prompt_builder import PromptBuilder
from haystack.components.generators import HuggingFaceLocalGenerator
from haystack.components.retrievers.in_memory import InMemoryBM25Retriever

# Build a RAG pipeline
TEMPLATE = '''
Given these documents, answer the question.
Documents:
{% for doc in documents %}
    {{ doc.content }}
{% endfor %}
Question: {{question}}
Answer:
'''

time_start = arrow.now()
pipeline = Pipeline()
pipeline.add_component('retriever', InMemoryBM25Retriever(document_store=document_store))
pipeline.add_component('prompt_builder', PromptBuilder(template=TEMPLATE))
pipeline.add_component('llm', HuggingFaceLocalGenerator())
pipeline.connect('retriever', 'prompt_builder.documents')
pipeline.connect('prompt_builder', 'llm')

print('built pipeline in {}'.format(arrow.now() - time_start))

2024-03-22 17:56:58.328254: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-03-22 17:56:58.328382: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-03-22 17:56:58.485271: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


built pipeline in 0:00:00.193103


In [5]:
# Ask a question
question = 'Who lives in Paris?'
results = pipeline.run(
    {
        'retriever': {'query': question},
        'prompt_builder': {'question': question},
    }
)

# print(results['llm']['replies'])
print(results['llm'])

config.json:   0%|          | 0.00/1.40k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/990M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/2.54k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

Ranking by BM25...:   0%|          | 0/3 [00:00<?, ? docs/s]

{'replies': ['Jean']}


/opt/conda/lib/python3.10/site-packages/transformers/generation/utils.py:1178: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
